In [1]:
using DelimitedFiles
using LinearAlgebra
using ForwardDiff
using Plots

## Function Definitions

In [13]:
# Data Type Format
#=
1.1 - name          <- Function name
2.1 - data          <- Data Matrix
3.1 - npts          <- Num. of points per sample 
4.1 - nout          <-
5.1 - model         <- Model function
6.1 - dim           <- Num. of parameters
7.1 - cluster       <-
8.1 - noise         <-
9.1 - solution      <- Vector Solution
10.1 - description  <-
=#

struct sample_type
    name::String
    data::Array{Float64,2}
    n::Int64
    dim::Int64
    model::Function
    solution::Vector{Float64}
end

function sample_parsing(file::String)
    data_sample = readdlm(file, ':')
    return sample_type(
                      data_sample[1,2],                   # name
                      eval(Meta.parse(data_sample[2,2])), # matrix
                      data_sample[3,2],                   # num pts
                      data_sample[6,2],                   # num parameters
                      eval(Meta.parse(data_sample[5,2])), # function
                      eval(Meta.parse(data_sample[9,2])), # solution
                      )
end

function dumping(f,R,J,dp::String)  
    DP1 = (norm(J'*R)^2)/(0.5*norm(R)^2)
    DP2 = norm(J'*R)^2
    DP3 = norm(J'*R)^2
    DP4 = 2*f
    DP4 = sqrt(2*f)
    #DP5 = (2*norm(J'*R)/(3*k)
    
    if dp=="DP1"
        return DP1
    elseif dp=="DP2"
        return DP2
    elseif dp=="DP3"
        return DP3
    elseif dp=="DP4"
        return DP4
    #elseif dp=="DP5"
    #    return DP5
    end
end

function Residual(A, t, n, dim, f)
    R = zeros(n)
    for i = 1 : n  
        R[i] = A[i,2]-f(A[i,1],t)
    end
    return R
end

"""
Goal: computes the Jacobian Matrix

#### Input:
     A - data matrix
     t - initial vector
     n - number of observations
     dim - dimension of model
     f - model to fit

#### Output:
     J - jacobian matrix
"""
function Jacobian(A, t, n, dim, f)
    J = zeros(n,dim)
    for i = 1:n
        r(t) = A[i,2]-f(A[i,1],t)
        J[i, :] = ForwardDiff.gradient(r,t)
    end
    return J
end


"""
Goal: 

#### Input:
     A - data matrix
     f - model function
     t - initial estimation
     dim - dim
     n - n
     lamb - Dumping parameter, may be a String or a Float
            if String: {}

#### Output:
     J - jacobian matrix
"""
function LM_dumping(A, f, t, dim::Int, n::Int, lamb=0.5, eps = 1.0e-9, itmax=1000)

    k = 0
    R = Residual(A,t,n,dim,f)
    J = Jacobian(A,t,n,dim,f)
    
    while norm(J'*R) > eps  && k < itmax
        
        if typeof(lamb)==String
            lamb = dumping(0.5*norm(R)^2,R,J,lamb)
        end
        # Verificar de o lambda esta alterando?
        
        M = J'*J + lamb*I
        d = M\(-J'*R)
        a = 1.0
        
        R_temp = Residual(A, t + a*d, n,dim,f)
        l = 0
        while 0.5*norm(R_temp)^2 > 0.5*norm(R)^2 + 0.5*a*(J'*R)'*d && l < itmax
            a = 0.5*a
            l = l +1
        end
        
        t = t +a*d
        k = k + 1
        
        R = Residual(A,t,n,dim,f)
        J = Jacobian(A,t,n,dim,f)
    end
    println(t,k)
    return t, k
end

LM_dumping

## Testing

In [11]:
sample_test = sample_parsing("/home/andrefz/PNL/problems-csv/parabola_-5.5_0.0_1.0_150_38.csv")

name = sample_test.name
A = sample_test.data
n = sample_test.n
dim = sample_test.dim
func = sample_test.model
sol = sample_test.solution


function cleaning(A, t, f)
    k = 0
    n, m = size(A)
    for i = 1:n
        if abs(A[i,2] - f(A[i,1],t)) < 1.0e-8 && abs(A[i,1]) > 1.0e-8  && abs(A[i,2]) > 1.0e-8
            k = k + 1
        end
    end
    
    M = zeros(k, m)
    d = 1
    for j = 1: n 
        if abs(A[j,2] - f(A[j,1],t)) < 1.0e-8 && abs(A[j,1]) > 1.0e-8  && abs(A[j,2]) > 1.0e-8
            M[d,:] = A[j,:]
            d = d + 1        
        end
    end
    
    return M
end

M = cleaning(A, sol, func)
LM_dumping(M, func, [0,0,0], dim, size(M)[1], "DP1")

#
# PASSANDO POR TODOS OS ARQUIVOS PARA VERIFICAR
#
# DIRETORIO ONDE ESTAO APENAS OS ARQUIVOS CSV
# AQUI VOCE COLOCA O DIRETORIO SO COM OS CSV

filelist = readdir("/home/andrefz/PNL/problems-csv/")
#for i=1:size(filelist)[1]
for i=1:3
    sample_test = sample_parsing("/home/andrefz/PNL/problems-csv/"*filelist[i])

    name = sample_test.name
    A = sample_test.data
    n = sample_test.n
    dim = sample_test.dim
    func = sample_test.model
    sol = sample_test.solution
    
    M = cleaning(A, sol, func)
    LM_dumping(M, func, sol, dim, size(M)[1], "DP1")
end

790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.593662

790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
790.5936629387619
[-5.499999999988416, 1.0316968168902018e-12, 0.9999999999766127]546
[-0.5, -3.5, -5.5, 1.5]0
[-0.5, -6.0, -2.5, 8.5]0
[-0.5, 2.0, -6.0, 5.0]0


## Ploting

In [4]:
aprox_solution, w = LM_dumping(A,func, [0.0,0.0,0.0], "DP1", dim, n)
aprox_matrix = copy(A)
typeof(aprox_solution)

for i=1:n
    aprox_matrix[i,2] = func(A[i,1], aprox_solution)
end

aprox_matrix = aprox_matrix[sortperm(aprox_matrix[:,1]),:]

# Plotando pontos
scatter(A[:,1],A[:,2], title="Cubica", label="model")
plot!(aprox_matrix[:,1],aprox_matrix[:,2], label="Aproximação")
xlabel!("x")
ylabel!("y")

LoadError: MethodError: no method matching LM_dumping(::Matrix{Float64}, ::var"#8#9", ::Vector{Float64}, ::String, ::Int64, ::Int64)
[0mClosest candidates are:
[0m  LM_dumping(::Any, ::Any, ::Any, [91m::Int64[39m, ::Int64, ::Any) at In[2]:102
[0m  LM_dumping(::Any, ::Any, ::Any, [91m::Int64[39m, ::Int64, ::Any, [91m::Any[39m) at In[2]:102
[0m  LM_dumping(::Any, ::Any, ::Any, [91m::Int64[39m, ::Int64, ::Any, [91m::Any[39m, [91m::Any[39m) at In[2]:102
[0m  ...